In [36]:
from pprint import pprint
from math import inf

class Graph:
    def __init__(self):
        self.nodes = {}

    def add_node(self, x, y, c='.', steps=0):
        if (x,y) not in self.nodes:
            self.nodes[(x, y)] = {'c': c, 'steps': steps, 'neighbors': []}

    def add_edge(self, node1, node2):
        self.nodes[node1]['neighbors'].append(node2)

    def node_exists(self, node):
        return node in self.nodes
        
def aoc23_10_proc_1a(fn,verbose=False):
    g0 = Graph()
    with open(fn, 'r') as file:
        lines = [s.strip() for s in file.readlines()]
    # Create the nodes
    for j, line in enumerate(lines):
        for i, c in enumerate(line):
            if c == 'S':
                g0.add_node(i,j,c,0)
                s_node = (i,j)
            elif c!='.':
                g0.add_node(i,j,c,inf)
  
    # Create the edges
    nb = {  '-': [(-1,0),(1,0)],
            '|': [(0,-1),(0,1)],
            '7': [(-1,0),(0,1)],
            'J': [(0,-1),(-1,0)],
            'L': [(0,-1),(1,0)],
            'F': [(0,1),(1,0)] }
    for j, line in enumerate(lines):
        for i, c in enumerate(line):
            if c in nb:
                for offset in nb[c]:
                    node = (i+offset[0],j+offset[1])
                    if g0.node_exists(node):
                         g0.add_edge((i,j),node)

    # Create the edges from S
    for offset in [(1,0),(0,1),(-1,0),(0,-1)]:
        node = (s_node[0]+offset[0],s_node[1]+offset[1])
        if g0.node_exists(node):
            if s_node in g0.nodes[node]['neighbors']:
                g0.add_edge(s_node,node)

    if (verbose): pprint(g0.nodes, sort_dicts=False)

    # Walk the graph
    g1 = Graph()
    prev_node = s_node
    g1.add_node(s_node[0],s_node[1],'S',0)
    cur_node = g0.nodes[s_node]['neighbors'][0]
    steps = 0
    while cur_node != s_node:
        g1.nodes[prev_node]['neighbors'] = [cur_node]
        steps += 1
        g1.add_node(cur_node[0],cur_node[1],g0.nodes[cur_node]['c'],steps)
        # Find where to go next
        for node in g0.nodes[cur_node]['neighbors']:
            if node != prev_node:   # Don't go back
                prev_node = cur_node
                cur_node = node
                break
    g1.nodes[prev_node]['neighbors'] = [cur_node]

    if verbose: pprint(g1.nodes, sort_dicts=False)

    # Walk the graph a different way
    g2 = Graph()
    prev_node = None
    cur_node = s_node
    g2.add_node(s_node[0],s_node[1],'S',0)
    steps = 0
    while steps==0 or cur_node != s_node:
        steps += 1
        # Find where to go next
        for node in g0.nodes[cur_node]['neighbors']:
            if node != prev_node:   # Don't go back
                g2.nodes[cur_node]['neighbors'] = [node]
                g2.add_node(node[0],node[1],g0.nodes[node]['c'],steps)
                prev_node = cur_node
                cur_node = node
                break
    g2.nodes[prev_node]['neighbors'] = [cur_node]

    if verbose: pprint(g2.nodes, sort_dicts=False)
    return (steps+1) // 2

In [37]:
fn = 'aoc23-10-test-1.txt'
res = aoc23_10_proc_1a(fn,True)
print(res)

{(1, 1): {'c': 'S', 'steps': 0, 'neighbors': [(2, 1), (1, 2)]},
 (2, 1): {'c': '-', 'steps': inf, 'neighbors': [(1, 1), (3, 1)]},
 (3, 1): {'c': '7', 'steps': inf, 'neighbors': [(2, 1), (3, 2)]},
 (1, 2): {'c': '|', 'steps': inf, 'neighbors': [(1, 1), (1, 3)]},
 (3, 2): {'c': '|', 'steps': inf, 'neighbors': [(3, 1), (3, 3)]},
 (1, 3): {'c': 'L', 'steps': inf, 'neighbors': [(1, 2), (2, 3)]},
 (2, 3): {'c': '-', 'steps': inf, 'neighbors': [(1, 3), (3, 3)]},
 (3, 3): {'c': 'J', 'steps': inf, 'neighbors': [(3, 2), (2, 3)]}}
{(1, 1): {'c': 'S', 'steps': 0, 'neighbors': [(2, 1)]},
 (2, 1): {'c': '-', 'steps': 1, 'neighbors': [(3, 1)]},
 (3, 1): {'c': '7', 'steps': 2, 'neighbors': [(3, 2)]},
 (3, 2): {'c': '|', 'steps': 3, 'neighbors': [(3, 3)]},
 (3, 3): {'c': 'J', 'steps': 4, 'neighbors': [(2, 3)]},
 (2, 3): {'c': '-', 'steps': 5, 'neighbors': [(1, 3)]},
 (1, 3): {'c': 'L', 'steps': 6, 'neighbors': [(1, 2)]},
 (1, 2): {'c': '|', 'steps': 7, 'neighbors': [(1, 1)]}}
{(1, 1): {'c': 'S', 'steps